In [ ]:
#mount our drive to load data 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#imports library 
import pandas as pd
import sklearn
import keras
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
import random
import os
import cv2 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import skimage 
from skimage import io,color
from skimage.io import imread,imshow
from skimage.color import rgb2gray,gray2rgb
from skimage.transform import  resize, rescale
from sklearn import datasets, metrics, model_selection,preprocessing
from sklearn.metrics import confusion_matrix,mean_squared_error,classification_report,ConfusionMatrixDisplay,roc_curve,roc_auc_score
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Flatten,Dense,Input
from keras.models import Model,Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,BatchNormalization, Activation
from pathlib import Path
from math import sqrt

In [ ]:
def load_dataset(PathOfSavedData):
  #load images as numpy array , and make splitting train, validation 
  X_train = np.load(PathOfSavedData+'X_train1.npy', mmap_mode='r')
  y_train = np.load(PathOfSavedData+'y_train1.npy', mmap_mode='r')
  X_valid = np.load(PathOfSavedData+'X_valid1.npy', mmap_mode='r')
  y_valid = np.load(PathOfSavedData+'y_valid1.npy', mmap_mode='r')

  
  X_train=X_train['arr_0']
  y_train=y_train['arr_0']
  X_valid=X_valid['arr_0']
  y_valid=y_valid['arr_0']

  return (X_train,y_train,X_valid,y_valid)
#call the load function
X_train,y_train,X_valid,y_valid = load_dataset("/content/drive/MyDrive/EmotionDataset/")

In [ ]:
#label encoding 
mapping = {"angry":0, "happy":1,"neutral":2, "sad":3, "surprise":4}
y_train_class = np.array([mapping[key] for key in y_train])
y_valid_class = np.array([mapping[key] for key in y_valid])

nClasses = 5
# Use Keras' handy utils
y_train_k = tensorflow.keras.utils.to_categorical(y_train_class, num_classes=nClasses)
y_valid_k = tensorflow.keras.utils.to_categorical(y_valid_class, num_classes=nClasses)

In [ ]:
#fisrt architecture   
def my_model():
  #start sequential model
  model= tf.keras.models.Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(92, 112,1)))
  model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
      
  model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten()) 
  model.add(Dense(256,activation = 'relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
      
  model.add(Dense(512,activation = 'relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))

  model.add(Dense(5, activation='softmax'))

  model.compile(
      optimizer = Adam(lr=0.0001), 
      loss='categorical_crossentropy', 
      metrics=['accuracy']
    )

  return model

In [ ]:
#second architecture
# def my_model(input_shape=(48,48,1)):
#     # first input model
#     visible = Input(shape=input_shape, name='input')
#     num_classes = 7
#     #the 1-st block
#     conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(visible)
#     conv1_1 = BatchNormalization()(conv1_1)
#     conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
#     conv1_2 = BatchNormalization()(conv1_2)
#     pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
#     drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)#the 2-nd block
#     conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
#     conv2_1 = BatchNormalization()(conv2_1)
#     conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
#     conv2_2 = BatchNormalization()(conv2_2)
#     conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
#     conv2_2 = BatchNormalization()(conv2_3)
#     pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
#     drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)#the 3-rd block
#     conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_1')(drop2_1)
#     conv3_1 = BatchNormalization()(conv3_1)
#     conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_2')(conv3_1)
#     conv3_2 = BatchNormalization()(conv3_2)
#     conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_3')(conv3_2)
#     conv3_3 = BatchNormalization()(conv3_3)
#     conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_4')(conv3_3)
#     conv3_4 = BatchNormalization()(conv3_4)
#     pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
#     drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)#the 4-th block
#     conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_1')(drop3_1)
#     conv4_1 = BatchNormalization()(conv4_1)
#     conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_2')(conv4_1)
#     conv4_2 = BatchNormalization()(conv4_2)
#     conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_3')(conv4_2)
#     conv4_3 = BatchNormalization()(conv4_3)
#     conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_4')(conv4_3)
#     conv4_4 = BatchNormalization()(conv4_4)
#     pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
#     drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1) 
#     #the 5-th block
#     conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_1')(drop4_1)
#     conv5_1 = BatchNormalization()(conv5_1)
#     conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_2')(conv5_1)
#     conv5_2 = BatchNormalization()(conv5_2)
#     conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_3')(conv5_2)
#     conv5_3 = BatchNormalization()(conv5_3)
#     conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_4')(conv5_3)
#     conv5_3 = BatchNormalization()(conv5_3)
#     pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
#     drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)#Flatten and output
#     flatten = Flatten(name = 'flatten')(drop5_1)
#     ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)# create model 
#     model = Model(inputs =visible, outputs = ouput)
#     # summary layers
#     print(model.summary())
    
#     return model


In [ ]:
model=my_model()
#summary of model 
model.summary()

In [ ]:
#checkpoint to save best weights  
checkpoint_filepath = 'model.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
#fit our model with batch size 64 with 20 epochs 
model.fit(X_train,
          y_train_k,
          batch_size=64,
          epochs=20,
          verbose=1,
          validation_data=(X_valid, y_valid_k),
          shuffle=True,
          callbacks=[model_checkpoint_callback]
          )

Epoch 1/20
728/728 [==============================] - 180s 228ms/step - loss: 7.4409 - accuracy: 0.5649 - val_loss: 5.9771 - val_accuracy: 0.6845
Epoch 2/20
728/728 [==============================] - 164s 225ms/step - loss: 4.8555 - accuracy: 0.7250 - val_loss: 3.8527 - val_accuracy: 0.7483
Epoch 3/20
728/728 [==============================] - 160s 220ms/step - loss: 3.0592 - accuracy: 0.7744 - val_loss: 2.3856 - val_accuracy: 0.7987
Epoch 4/20
728/728 [==============================] - 160s 220ms/step - loss: 1.9682 - accuracy: 0.8044 - val_loss: 1.6298 - val_accuracy: 0.8124
Epoch 5/20
728/728 [==============================] - 160s 219ms/step - loss: 1.3587 - accuracy: 0.8262 - val_loss: 1.2028 - val_accuracy: 0.8162
Epoch 6/20
728/728 [==============================] - 160s 219ms/step - loss: 1.0042 - accuracy: 0.8455 - val_loss: 1.0308 - val_accuracy: 0.8115
Epoch 7/20
728/728 [==============================] - 160s 219ms/step - loss: 0.8354 - accuracy: 0.8525 - val_loss: 0.8553 -

In [ ]:
#save model 
model.save('CNN-Arch.h5')